# Production Schedule Optimization for CFP
### 2020-12-09
### D. Garson

Purpose of the program is to compute the cost minimizing production allocation & schedule for multiple products produced across muliple plants. Designed for proof of concept app on Corporate Finance Platform. This formulation is a simplified version based on an AMPL script that combined production allocation and scheduling. It uses Google OR Tools to solve the problem in 2 phases: 
1. allocating production quantities to plants given global demand targets
2. schedule blocks of production within each plant

In [ ]:
import ortools, pandas as pd, numpy as np

### Phase 1: Production Allocation

Given overall production targets over X **operational** hours, minimize the cost of production across all plants subject to constraints on production capabilities at each plant. 

**Drivers of cost:** 
1. cost of raw materials
2. cost of labour per hour 
3. cost of changeover (broadly considered here using avg. plant changeover cost * # products allocated)
4. Fixed operating cost of bakery to be operational for X hour

**Constraints:**
1. Minimum batch size per product at each plan must be satifsied, else 0 production
2. Every plant only has X hours of production, after a downtime and maintenance adjustment is applied. (productive time + changeover time = x operational hours, user defined)
3. throughput of each product at each bakery determines the productivity per hour

**Assumptions**
1. Minimum batch size determines the incriments products are produced in
2. All production targets must be met, and left over goods are considered waste
3. Cost of waste is determined by the production costs of unused products

In [ ]:
## helper fn to consolidate production specific info at each bakery

def append_production_params(production_params):
    
    # list of dfs per plant
    p_list = list(production_params.keys())
    first = p_list.pop(0)
   
    # first plant's df
    df = production_params[first]
    
    
    for i in p_list:
        df = df.append(production_params[i], ignore_index=True)
    
    return(df)

In [ ]:
# read raw data and concatenate master data to be used in calculating coef values
path = "C:/Users/TH845UT/OneDrive - EY/Desktop/Proof of Concept/forecast & schedule model/"
file = "MASTER_FFP_DATA_TEMPLATE.xlsx"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# information at plant level (labour rate, productive hours available, paid downtime adj.)
global_plant_params = pd.read_excel(path+file,sheet_name = 'Plant Params')

# information specific to producing each product at each bakery
production_params_tmp = pd.read_excel(path+file,sheet_name = ['Plant1','Plant2','Plant3','Plant4','Plant5','Plant6'],usecols=[0,1,2,3,4,5])
production_params = append_production_params(production_params_tmp)

# information on every FROM->TO product changeover time
changeover_params = pd.read_excel(path+file,sheet_name = 'Changeover Params')

# information on global demanded qunatity of each product
product_params = pd.read_excel(path+file,sheet_name = 'Product Params')





In [ ]:
## Indexing convention on decision variable X[p,i] = number of hours spent in production of product I at plant P
## Plant and Product Indicies goes in ascending order
## i.e. coef * X[plant1,product1]  + coef * X[plant1,product2]  + ... 
## ...  coef * X[plant6,product59] + coef * X[plant6,product60] 

## index order will follow the order of the master vectors below:

PLANTS = production_params.Plant.unique()
N_PLANTS = len(PLANTS)


PRODUCTS = product_params.Product.unique()
N_PRODUCTS = len(PRODUCTS)


## order production_params and changeover_params to conform to this ordering convention
production_params['product_ix'] = production_params.Product.str[1:]
production_params["product_ix"] = production_params["product_ix"].apply(pd.to_numeric)
production_params = production_params.sort_values(['Plant', 'product_ix'],ascending=[True,True])

## remove records where product_params have products not listed in PRODUCTS master list
ix1 = production_params.Product.isin(PRODUCTS)
production_params = production_params[ix1]


#changeover_params.sort_index(level=player_order)
changeover_params['from_product_ix'] = changeover_params.FromProduct.str[1:]
changeover_params['to_product_ix'] = changeover_params.ToProduct.str[1:]
changeover_params["from_product_ix"] = changeover_params["from_product_ix"].apply(pd.to_numeric)
changeover_params["to_product_ix"] = changeover_params["to_product_ix"].apply(pd.to_numeric)
changeover_params = changeover_params.sort_values(['Plant', 'from_product_ix','to_product_ix'],ascending=[True,True,True])

## remove records where changeover_params have products not listed in PRODUCTS master list
ix2 = changeover_params.ToProduct.isin(PRODUCTS) & changeover_params.FromProduct.isin(PRODUCTS)
changeover_params = changeover_params[ix2]


# count of products per plant (after removing unused products)
PRODUCTS_PER_PLANT = production_params[['Plant','Product']].groupby('Plant').count()




#### Constructor for compontents of OBJECTIVE FUNCTION
(1) Material Costs

(2) Labour Costs

(3) Changeover Cost (estmiate)

(4) Fixed Plant Operating Costs

(5) Cost paid downtime & maintenance (constant)

In [ ]:
# (1) Material Costs
# SUM { X[Plant i, Product j] * Throughput [Plant i, Product j] * Unit_Material_Cost[Plant i,Product j] }
def obj_constr_material_cost():
    
    # array for material cost coefficients 
    material_cost_coefs = np.array(production_params.throughput * production_params.unit_material_cost) 
    
    return(list(material_cost_coefs))
    

In [ ]:
# (2) Labour Costs
#  SUM { X[Plant i, Product j] * Labour_Rate[Plant i] * Crew_Size[Plant i,Product j]
def obj_constr_labour_cost():
    
    # table showing count of products that can be made at each plant
    # prod_per_plant = production_params[['Plant','Product']].groupby(['Plant']).count().Product
    
    # repeating labour rate, so that coeficient is conformable with mat dimensions
    lr_rep = list(np.repeat(global_plant_params.LabourRate,PRODUCTS_PER_PLANT.Product))  
    
    # crew size required
    crew_req = list(production_params.labour_req)
        
    # coef array, in correct order convention
    n = PRODUCTS_PER_PLANT.Product.sum()
    labour_cost_coefs = [lr_rep[i] * crew_req[i] for i in range(n)]
    
    return(labour_cost_coefs)


In [ ]:
# (3) Changeover Cost (estmiate)
# SUM { X[Plant i, Product j] * Average_Changeover_cost_TO_and_FROM[Plant i] 
def obj_constr_changeover_cost():
    
    # pre-allocate avg changeover time per product at each bakery
    avg_changeover = production_params[['Plant','Product']]
    avg_changeover.loc[:,'avg_cost'] = 0
    
    # count of products per plant
    # prod_count = production_params[['Plant','Product']].groupby('Plant').count()
    
    
    for p in PLANTS:
        
        # labour rate at plant
        ix3 = global_plant_params['Plant'] == p
        
        for prod in PRODUCTS:
            
            # boolean indexing, for every plant for every product (where product is either one of TO or FROM)
            ix = (changeover_params['Plant'] == p) & ((changeover_params['FromProduct'] == prod) | (changeover_params['ToProduct'] == prod))
            
            # plant & product index to insert calculated changeover cost into
            ix2 = (avg_changeover['Plant'] == p) & (avg_changeover['Product'] == prod)
            
            # labour rate at plant
            lr = global_plant_params.loc[ix3,"LabourRate"]
            
            # insert changeover cost per plant and product
            m = changeover_params[ix].changeover_time_hrs.mean()
            
            avg_changeover.loc[ix2,'avg_cost'] =  (m * float(lr))

            # average changeover cost vector for objective function
            # this series of variables represents the binary variables, switched on = 1, when 
            # minimum quantity threshold is reached for that bakery-product combo:
            # Plants ascending, product ascending
            # order: coef*Y[plant1,prod1] + coef*Y[plant1,prod2] + .... + coef*Y[plant6,prod50]

    return (list(avg_changeover.avg_cost))
            
            


In [ ]:
# (4) Fixed Plant Operating Costs
#     binary integer variables, one per plant, ctivated when plant produces non-zero quantity

def obj_constr_fixed_cost():
    
    return(list(global_plant_params.FixedCost))
    
    

In [ ]:
# (4) Cost paid downtime & maintenance (constant)
#     accounting for paid time that is non-productive, based on MD_adj percentage. Labour is paid during this period
def obj_constr_constant():
    
    avg_crew_size = production_params[["Plant","labour_req"]].groupby("Plant").mean()
   
    v = [global_plant_params.FullUtilizationHrs[i] * global_plant_params.MD_adj[i] * global_plant_params.LabourRate[i] * avg_crew_size.labour_req[i] for i in range(N_PLANTS)]
    
    sum_v = sum(v)
    
    return (sum_v)



#### Order of variables in objective fn vector:

1. variables representing the # hours spent in producing product x and plant p  ( plants * products, coefs = labour + material) costs)
2. variables representing whether production of each good occured at each bakery (plant * products, coesf = changeover cost)
3. variables representing binary plant activation (# plants)
4. constant representing cost of downtime/maintenance adj for all plant (1)  (**NOTE** no need to include constant in col index)

In [ ]:
# 6 plants, 50 products = 606 variables, indexed [0,605]
N_COL_CONSTR_MAT = PRODUCTS_PER_PLANT.Product.sum() * 2 + N_PLANTS


## Constraint Matrix Constructors

note that # columns in the constraint matrix corresponds to the order of the obj. fn variables

1. global demand targets must be met. 
2. productive hours per plant + estimated total changeover cannot exceed [ProductionHours] ( full utilization hrs * (1 - MD_adj) ) 
3. activation constraints on binary indicator varibles indexed by {Plant, Product} 
4. activation constraints on binary indicator variables indexed by {Plant}

In [ ]:
# (1) Global demand targets must be met
# number of constraints = # products

## APPLIES TO ACTIVATING COEFS FOR COLUMN INDEXES (0,300)
## only the variables that represent "# hours of production of product m at bakery n" are activated
# in constraint (1)

# thoughput[plant1,prod1] * X[plant1,prod1] + ... + thoughput[plantN,prod1] * X[plantN,prod1] >= GlobalProductionTarget[prod1]
# thoughput[plant1,prod2] * X[plant1,prod2] + ... + thoughput[plantN,prod2] * X[plantN,prod2] >= GlobalProductionTarget[prod2]
# ...
# thoughput[plant1,prodM] * X[plant1,prodM] + ... + thoughput[plantN,prodM] * X[plantN,prodM] >= GlobalProductionTarget[prodM]

def constraint_demand_targets():
    
    # construct starting matrix, matrix of throughputs dimensions: (# products) x (# bakery-product combos)
    # take production_params.througput, transpose and repeat by (# products) for rows
    a = np.array(production_params.throughput)
    
    # construct matrix of indicator variables corresponding to the 
    # dimensions of throughput_mat. For each row, only coefs corresponding 
    # to productN will be = throughput coef value, otherwise = 0
    constr_mat = np.zeros([N_PRODUCTS,N_COL_CONSTR_MAT])
    ix_1 = [np.where(production_params.Product == p ) for p in PRODUCTS]
    
    for r in range(N_PRODUCTS):
        constr_mat[r,ix_1[r]] =  a[ix_1[r]]
        
    # adjustments made to constr_mat to conform with 
    # overall dimensions of A matrix (argmin x: Ax=b)
    # keeping in mind the indexing convension. all other index positions assigned coef = 0
    
    # 1. variables representing the # hours spent in producing product x and plant p  ( plants * products, coefs = labour + material) costs)
    # 2. variables representing whether production of each good occured at each bakery (plant * products, coesf = changeover cost)
    # 3. variables representing binary plant activation (# plants)
    # 4. constant representing cost of downtime/maintenance adj for all plant (1) <-- don't include constant in [A] column dimensions
      
       
    # Right Hand Side
    rhs = product_params.GlobalQuantity
    
    return constr_mat.tolist(), rhs.tolist()
    




In [ ]:
## (1A) # get average changeover times, ordered by Plant asc, product asc
## THIS IS A HELPER FUNCTION, not a constraint

def coefs_avg_changeover_time():
    
    # pre-allocate avg changeover time per product at each bakery
    avg_changeover = production_params[['Plant','Product']]
    avg_changeover.loc[:,'avg_time'] = 0
    
    # count of products per plant
    # prod_count = production_params[['Plant','Product']].groupby('Plant').count()
    
    
    for p in PLANTS:
        
        # labour rate at plant
        ix3 = global_plant_params['Plant'] == p
        
        for prod in PRODUCTS:
            
            # boolean indexing, for every plant for every product (where product is either one of TO or FROM)
            ix = (changeover_params['Plant'] == p) & ((changeover_params['FromProduct'] == prod) | (changeover_params['ToProduct'] == prod))
            
            # plant & product index to insert calculated changeover cost into
            ix2 = (avg_changeover['Plant'] == p) & (avg_changeover['Product'] == prod)
            
            # labour rate at plant
            # lr = global_plant_params.loc[ix3,"LabourRate"]
            
            # insert changeover cost per plant and product
            m = changeover_params[ix].changeover_time_hrs.mean()
            
            avg_changeover.loc[ix2,'avg_time'] =  m 
            
            # average changeover cost vector for objective function
            # this series of variables represents the binary variables, switched on = 1, when 
            # minimum quantity threshold is reached for that bakery-product combo:
            # Plants ascending, product ascending
            # order: coef*Y[plant1,prod1] + coef*Y[plant1,prod2] + .... + coef*Y[plant6,prod50]

    return (avg_changeover.avg_time)
            

In [ ]:
# (2) productive hours per plant + estimated changeover time cannot exceed [ProductionHours] ( full utilization hrs * (1 - MD_adj) )

## APPLIES TO ACTIVATING COEFS FOR COLUMN INDEXES (0 - 299) AND (300 - 599 )
## only the variables that represent "# hours of production of product m at bakery n"
## and "binary indicator variables representing positive production of product m at bakery n" are activated
# in constraint (2)

# number of constraints = # plants: 

# 1 * X[plant1,prod2] + ... + 1* X[plant1,prodM] + ... + Y[plant1,prod1] + Y[plant1,prod2] +... + Y[plant1,prodM] <= ProductionHours[plant1]
# 1 * X[plant2,prod2] + ... + 1* X[plant2,prodM] + ... + Y[plant2,prod1] + Y[plant2,prod2] +... + Y[plant2,prodM] <= ProductionHours[plant2]
# .... 
# 1 * X[plantN,prod2] + ... + 1* X[plantN,prodM] + ...+ Y[plantN,prod1] + Y[plantN,prod2] +... + Y[plantN,prodM]  <= ProductionHours[plantN]
  

def constraint_max_hours():
    
    # pre-allocate dimensions of constraint matrix
    constr_mat = np.zeros([N_PLANTS,N_COL_CONSTR_MAT])
    
    # first index of 1s coefs
    ix_1 = [np.where(production_params.Plant == p ) for p in PLANTS]
    
    # second index of avg changeover times
    ix_2 = ix_1 + PRODUCTS_PER_PLANT.Product.sum()
    
    # average changeover costs
    ch = coefs_avg_changeover_time()
    
    for r in range(N_PLANTS):
        constr_mat[r,ix_1[r]] =  1
        i = ix_1[r]
        constr_mat[r,ix_2[r]] = ch[i[0]]
        
        
    # right hand side
    rhs = global_plant_params.ProductionHours
    
    
    return constr_mat.tolist(), rhs.tolist()
    

In [ ]:
# 3. activation constraints on binary indicator varibles indexed by {Plant, Product} 
# positive quantity is produced of productM at plantN only if minimum quantity threshold is reached

## FROM AMPL: minimum batch quantity must be reached for each product at each bakery, 
#      if non zero quanity is produced.
# subject to min_batch_size_A {b in BAKERY, p in PRODUCT[b]}: 
#      t[b,p]*throughput[b,p] >= min_batch_q[b,p]*min_q[b,p];

#  number of constrains = N_PLANTS * N_PRODUCTS
# --> X[plant1,product1] - min_batch_q[plant1,product1]* Y[plant1,product1] >= 0 
#     X[plant2,product1] - min_batch_q[plant2,product1]* Y[plant2,product1] >= 0 
#     ....
#     X[plantN,productM] - min_batch_q[plantN,productM]* Y[plantN,productM] >= 0 
 

# activates coeficients at indexes (0,299) and (300,599)
def constraint_min_batch_q_A():

    n = PRODUCTS_PER_PLANT.Product.sum()

    constr_mat = np.zeros([n,N_COL_CONSTR_MAT])
    
    for i in range(n):
        constr_mat[i,i] = 1
        constr_mat[i,i+n] = -production_params.min_batch_size[i]
        
    # rhs
    rhs = np.zeros([n,1])

    return constr_mat.tolist(), rhs.tolist()
        
    
    

# subject to min_batch_size_B {b in BAKERY, p in PRODUCT[b]}: 
#         t[b,p] <= ProductionHoursPerBakery[b] * min_q[b,p]; 

#  number of constrains = N_PLANTS * N_PRODUCTS
# --> X[plant1,product1] - ProductionHours[plant1]* Y[plant1,product1] <= 0 
#     X[plant2,product1] - ProductionHours[plant2]* Y[plant2,product1] <= 0 
#     ....
#     X[plantN,productM] - ProductionHours[plantN]* Y[plantN,productM] <= 0 

def constraint_min_batch_q_B():

    n = PRODUCTS_PER_PLANT.Product.sum()

    constr_mat = np.zeros([n,N_COL_CONSTR_MAT])
    
    for i in range(n):
        p = production_params.Plant[i]
        constr_mat[i,i] = 1
        v = round(float(global_plant_params.loc[global_plant_params["Plant"]==p].ProductionHours)*(-1),2)
        constr_mat[i,i+n] = v
        
    # rhs
    rhs = np.zeros([n,1])
    
    return constr_mat.tolist(), rhs.tolist()




In [ ]:
# 4. activation constraints on binary indicator variables indexed by {Plant}

# number of constraints = number of plants
# Y[plant1,prod1] + Y[plant1,prod2] + Y[plant1,prod3] + ... + Y[plant1,prodM] <= 100 * P[plant1] 
# Y[plant2,prod1] + Y[plant2,prod2] + Y[plant2,prod3] + ... + Y[plant2,prodM] <= 100 * P[plant2]
# ...
# Y[plantN,prod1] + Y[plantN,prod2] + Y[plantN,prod3] + ... + Y[plantN,prodM] <= 100 * P[plantN] 

# impacts coefs on indexes (300,599) and (600,606)

def constraint_plant_activation():
    
    n = PRODUCTS_PER_PLANT.Product.sum()
    
    constr_mat = np.zeros([N_PLANTS,N_COL_CONSTR_MAT])
    
    # first index of 1s coefs
    ix_1 = [np.where(production_params.Plant == p ) for p in PLANTS]
    ix_1 = ix_1 + n
    
    # second index of avg changeover times
    ix_2 = range(2*n,2*n + N_PLANTS)
    
    for p in range(N_PLANTS):
        constr_mat[p,ix_1[p]] = 1
        constr_mat[p,ix_2[p]] = -100
        
    # RHS
    rhs = np.repeat(0,N_PLANTS)
    
    return constr_mat.tolist(), rhs.tolist()
    
    
    
    


### use constructors to build optimization objects in required formats for Google OR Tools package


In [ ]:
## sample code from:  https://developers.google.com/optimization/mip/mip_var_array

import ortools
from ortools.linear_solver import pywraplp


def create_data_model():

    data = {}
    
    ## OBJECTIVE COEFICIENTS
    # order Plant asc, Product asc 
    # [Production time vars: Plant * Products (50 * 6 = 300)] [Production binary Plant * Products (50 * 6 = 300)][Plant binary (# Plants) 6]
    # [ indicies 0 - 299] [indicies 300 - 599] [indicies 600-605]
    
    # (1) [Production time vars: Plant * Products (50 * 6 = 300)]
    material_cost_coefs = obj_constr_material_cost()
    labour_cost_coefs =  obj_constr_labour_cost()
    n = PRODUCTS_PER_PLANT.Product.sum()
    
    obj_1 = [material_cost_coefs[i] + labour_cost_coefs[i] for i in range(n)]
    
    # (2)[Production binary Plant * Products (50 * 6 = 300)]
    obj_2 = obj_constr_changeover_cost()
    
    # (3) [Plant binary (# Plants) 6]
    obj_3 = obj_constr_fixed_cost()
    
    # (4) objective constant (labour cost of )
    obj_constant_term = obj_constr_constant()
    
    
    data['obj_coeffs'] = obj_1 + obj_2 + obj_3 
    
    
    
    ## CONSTRAINTS
    
    
    # (1) Global demand targets must be met
    # number of constraints = # products

    ## APPLIES TO ACTIVATING COEFS FOR COLUMN INDEXES (0,300)
    ## only the variables that represent "# hours of production of product m at bakery n" are activated
    # in constraint (1)

    # thoughput[plant1,prod1] * X[plant1,prod1] + ... + thoughput[plantN,prod1] * X[plantN,prod1] >= GlobalProductionTarget[prod1]
    # thoughput[plant1,prod2] * X[plant1,prod2] + ... + thoughput[plantN,prod2] * X[plantN,prod2] >= GlobalProductionTarget[prod2]
    # ...
    # thoughput[plant1,prodM] * X[plant1,prodM] + ... + thoughput[plantN,prodM] * X[plantN,prodM] >= GlobalProductionTarget[prodM]

    c_1, rhs_1 = constraint_demand_targets()
    
    
    
    # (2) productive hours per plant + estimated changeover time cannot exceed [ProductionHours] ( full utilization hrs * (1 - MD_adj) )

    ## APPLIES TO ACTIVATING COEFS FOR COLUMN INDEXES (0 - 299) AND (300 - 599 )
    ## only the variables that represent "# hours of production of product m at bakery n"
    ## and "binary indicator variables representing positive production of product m at bakery n" are activated
    # in constraint (2)

    # number of constraints = # plants: 

    # 1 * X[plant1,prod2] + ... + 1* X[plant1,prodM] + ... + Y[plant1,prod1] + Y[plant1,prod2] +... + Y[plant1,prodM] <= ProductionHours[plant1]
    # 1 * X[plant2,prod2] + ... + 1* X[plant2,prodM] + ... + Y[plant2,prod1] + Y[plant2,prod2] +... + Y[plant2,prodM] <= ProductionHours[plant2]
    # .... 
    # 1 * X[plantN,prod2] + ... + 1* X[plantN,prodM] + ...+ Y[plantN,prod1] + Y[plantN,prod2] +... + Y[plantN,prodM]  <= ProductionHours[plantN]
    
    c_2, rhs_2 = constraint_max_hours()
    
    
    # 3. activation constraints on binary indicator varibles indexed by {Plant, Product} 
    # positive quantity is produced of productM at plantN only if minimum quantity threshold is reached

   ## FROM AMPL: minimum batch quantity must be reached for each product at each bakery, 
   #      if non zero quanity is produced.
   # subject to min_batch_size_A {b in BAKERY, p in PRODUCT[b]}: 
   #      t[b,p]*throughput[b,p] >= min_batch_q[b,p]*min_q[b,p];

    #  number of constrains = N_PLANTS * N_PRODUCTS
    #  X[plant1,product1] - min_batch_q[plant1,product1]* Y[plant1,product1] >= 0 
    #  X[plant2,product1] - min_batch_q[plant2,product1]* Y[plant2,product1] >= 0 
    #     ....
    #  X[plantN,productM] - min_batch_q[plantN,productM]* Y[plantN,productM] >= 0 
    
    c_3, rhs_3 = constraint_min_batch_q_A()
        

    # subject to min_batch_size_B {b in BAKERY, p in PRODUCT[b]}: 
    # t[b,p] <= ProductionHoursPerBakery[b] * min_q[b,p]; 

    #  number of constrains = N_PLANTS * N_PRODUCTS
    #  X[plant1,product1] - ProductionHours[plant1]* Y[plant1,product1] <= 0 
    #  X[plant2,product1] - ProductionHours[plant2]* Y[plant2,product1] <= 0 
    #    ....
    #  X[plantN,productM] - ProductionHours[plantN]* Y[plantN,productM] <= 0 

    c_4, rhs_4 = constraint_min_batch_q_B()     
    
    
    # 4. activation constraints on binary indicator variables indexed by {Plant}

    # number of constraints = number of plants
    # Y[plant1,prod1] + Y[plant1,prod2] + Y[plant1,prod3] + ... + Y[plant1,prodM] <= 100 * P[plant1] 
    # Y[plant2,prod1] + Y[plant2,prod2] + Y[plant2,prod3] + ... + Y[plant2,prodM] <= 100 * P[plant2]
    # ...
    # Y[plantN,prod1] + Y[plantN,prod2] + Y[plantN,prod3] + ... + Y[plantN,prodM] <= 100 * P[plantN] 

    # impacts coefs on indexes (300,599) and (600,606)

    c_5, rhs_5 = constraint_plant_activation()
    
    
    # plant-product combos
    n = PRODUCTS_PER_PLANT.Product.sum()
    
    data['constraint_coeffs'] = c_1 + c_2 + c_3 + c_4 + c_5
    data['bounds'] = rhs_1 + rhs_2 + rhs_3 + rhs_4 + rhs_5 
    
    bounds = data['bounds']
    flatten_bounds = list()
    for b in bounds:
        if type(b) == list:
            flatten_bounds.append(b.pop())
        else:
            flatten_bounds.append(b)
    
    data['bounds'] = flatten_bounds

    data['num_vars'] = N_COL_CONSTR_MAT
    data['num_constraints'] = N_PLANTS*2 + N_PRODUCTS + n*2 
    
    return data




In [ ]:
def main():
    
    data = create_data_model()

 

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')  ## see other solver options "SCIP","GLOP"
    infinity = solver.infinity()
    
    x = {}
    
    # define which variables in solution vector X are continuous vs integer
    # relying on the indexing defined inherently in the order the problem was constructed

    # indicies [0,299] are continuous, Production_Hours{plant N, Product M}
    # indicies [300,599] are binary indicator vars, Production_Activated{Plant N, Product M} 
    # indicies [600,605] are binary indicator vars, Plant_Activated{Plant N}
    
    n = PRODUCTS_PER_PLANT.Product.sum()
    
    ix_1_range = range(n) # continuous vars [0,Inf] --> default (?)
    ix_2_range = range(n,n*2+N_PLANTS) # binary integer vars [0,1]
    
    print('Number of continuous variables =', n) 
    for j in ix_1_range:
        x[j] = solver.NumVar(0, infinity, 'x[%i]' % j)
       
    
    print('Number of binary integer variable =', len(ix_2_range))    
    for j in ix_2_range:
        x[j] = solver.IntVar(0, 1, 'x[%i]' % j)
        
        
    ##  defining the numer of row constraints 
    # for i in range(data['num_constraints']):
    #    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    #    for j in range(data['num_vars']):
    #        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
    #        print('Number of constraints =', solver.NumConstraints())
  
    # In Python, you can also set the constraints as follows. NOTE: SIGN CHANGES
    
    # which constraints have which sign (>=, = ,<=)
    constraint_s1 = range(N_PRODUCTS) # constr >= rhs, row indicies [0,49]  # products
    constraint_s2 = range(N_PRODUCTS,N_PRODUCTS+N_PLANTS) # constr <= rhs, row indicies [50,55] # plants
    constraint_s3 = range(N_PRODUCTS+N_PLANTS,N_PRODUCTS+N_PLANTS+n)# constr >= rhs, row indicies [56,355]
    constraint_s4 = range(N_PRODUCTS+N_PLANTS+n,N_PRODUCTS+N_PLANTS+n*2) # constr <= rhs, row indicies [356,655]
    constraint_s5 = range(N_PRODUCTS+N_PLANTS+n*2,N_PRODUCTS+N_PLANTS*2+n*2)# constr <= rhs, row indicies [656,661]
   
    constr_greater = list(constraint_s1) + list(constraint_s3)
    constr_lesser = list(constraint_s2) + list(constraint_s4) + list(constraint_s5)
    
    print("Constructing " + str(len(constr_lesser)) + " lesser-equal <= constraints")
    for i in constr_lesser:
        constraint_expr = \
        [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
        solver.Add(sum(constraint_expr) <= data['bounds'][i])
    
    print("Constructing " + str(len(constr_greater)) + " greater-equal >= constraints")
    for i in constr_greater:
        constraint_expr = \
        [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
        solver.Add(sum(constraint_expr) >= data['bounds'][i])
     
  
    # OBJECTIVE COEFICIENTS  
    print("Constructing objective coefficients")
    objective = solver.Objective()
    for j in range(data['num_vars']):
        objective.SetCoefficient(x[j], data['obj_coeffs'][j])
        objective.SetMinimization()
  
    
    ## SOLVE 
    status = solver.Solve()

    ## SOLVE OUTPUT 
    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        
    for j in range(data['num_vars']):
        print(x[j].name(), ' = ', x[j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    
    else:
        print('The problem does not have an optimal solution.')




In [ ]:
main()